# Segmantation

В этом занятии мы рассмотрим полный цикл подготовки модели нейронной сети для решения задачи сегментации на примере задачи [Carvana Challange](https://www.kaggle.com/c/carvana-image-masking-challenge).

[Данные](https://drive.google.com/file/d/13_atfxCGnS7Qs3WYk_1h5-3RXaO5-efc/view?usp=sharing) для семинара.

__В этом занятии будет:__

__1. [Подготовка данных](#1.-Подготовка-данных)__

__2. [Сборка модели](#2.-Сборка-модели)__

__3. [Тренировка](#3.-Тренировка)__

__4. [Аугментация](#4.-Аугментация)__

__5. [Try Hard](#5.-Try-Hard)__

In [1]:
# install new packages
!pip install albumentations
!pip install opencv-python

In [3]:
# data processing
import os
import gc
import cv2
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline
warnings.filterwarnings("ignore")

# nn modules
import keras
from keras import backend as K
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import (
    Dense,
    Input,
    GlobalAveragePooling2D, 
    Dropout, 
    UpSampling2D, 
    Conv2D, 
    MaxPooling2D,
    Concatenate,
    Activation
)

from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D

ModuleNotFoundError: No module named 'tensorflow'

## 1. Подготовка данных 

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
        
    img = img.reshape(shape)
    return img

In [ ]:
df = pd.read_csv('input/train_masks.csv')
print(f'Data shape: {df.shape}')
df.head()

In [ ]:
# make train/val dataset
all_id = np.array(range(df.shape[0]), dtype=np.uint8)
np.random.seed(42)
np.random.shuffle(all_id)

split_size = 0.8
train_df = df.loc[all_id[:int(df.shape[0] *split_size)]]
val_df = df.loc[all_id[int(df.shape[0] * split_size):]]

print(f'Train DataFrame shape: {train_df.shape}\n'
     f'Validation DataFrame shape: {val_df.shape}')

In [ ]:
ex_num = 10
random_id = np.random.choice(train_df.shape[0], size=ex_num, replace=False)

for i, row_id in enumerate(random_id):
    img_name, mask_rle = train_df.iloc[row_id]
    img = cv2.imread('input/train/{}'.format(img_name))
    w, h, _ = img.shape
    mask = rle_decode(mask_rle, shape=(w, h, 1))
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25, 25))
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # change colorspace for plt
    axes[1].imshow(mask[..., 0], cmap='gray')
    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

### Train genrator 

In [ ]:
def keras_generator(gen_df, batch_size=4):
    while True:
        x_batch = []
        y_batch = []
        
        random_id = np.random.choice(gen_df.shape[0], size=batch_size, replace=False)
        for row_id in random_id:
            img_name, mask_rle = gen_df.iloc[row_id]
            img = cv2.imread('input/train/{}'.format(img_name))
            w, h, _ = img.shape
            mask = rle_decode(mask_rle, shape=(w, h, 1))

            img = cv2.resize(img, (256, 256))
            mask = cv2.resize(mask, (256, 256))
            
            x_batch += [img]
            y_batch += [mask]

        x_batch = np.array(x_batch) / 255.  # normalize
        y_batch = np.array(y_batch)

        yield x_batch, np.expand_dims(y_batch, -1)

In [ ]:
# generator testing
train_gen = keras_generator(train_df, 10)
x, y = next(iter(train_gen))

print(f'Input size: {x.shape}\n'
     f'Output size: {y.shape}')

## 2. Сборка модели

Начнем собирать первую модель для решения задачи сегментации на основе FCN
<img src="https://i.ibb.co/TMwThH1/fcn.jpg" width="800">

In [ ]:
vgg16_model = VGG16(weights='imagenet', input_shape=(256,256,3), include_top=False)
up = UpSampling2D(32, interpolation='bilinear')(vgg16_model.output)
conv = Conv2D(1, (1, 1))(up)
conv = Activation('sigmoid')(conv)

fcn_model = Model(input=vgg16_model.input, output=conv)

In [ ]:
# посмотрим на архитекутуру сети
fcn_model.summary()

In [ ]:
# тест входных и выходных размерностей модели
test_array = np.zeros((10, 256, 256, 3), dtype='float32')
output = fcn_model(K.variable(test_array))

assert K.eval(output).shape[:3] == test_array.shape[:3], f'Incorect output shape: {K.eval(output).shape} \
when input shape: {test_array.shape}'

In [ ]:
best_w = keras.callbacks.ModelCheckpoint('fcn_best.h5',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=True,
                                save_weights_only=True,
                                mode='auto',
                                period=1)

last_w = keras.callbacks.ModelCheckpoint('fcn_last.h5',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=False,
                                save_weights_only=True,
                                mode='auto',
                                period=1)
callbacks = [best_w, last_w]

adam = keras.optimizers.Adam(lr=0.1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
fcn_model.compile(adam, 'binary_crossentropy')

## 3. Тренировка 

In [ ]:
batch_size = 4
train_gen = keras_generator(train_df, batch_size)
val_gen = keras_generator(val_df, batch_size)

hist = fcn_model.fit_generator(
    train_gen,
    steps_per_epoch=100,
    epochs=5,
    callbacks=callbacks,
    validation_data=val_gen,
    validation_steps=50,
    class_weight=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    shuffle=True,
    initial_epoch=0,
    verbose=1,)

In [ ]:
def plot_predicts(model, ex_num=5, th_pred=0.5):
    random_id = np.random.choice(val_df.shape[0], size=ex_num, replace=False)

    for i, row_id in enumerate(random_id):
        img_name, mask_rle = train_df.iloc[row_id]
        img = cv2.imread('input/train/{}'.format(img_name))
        w, h, _ = img.shape
        true_mask = rle_decode(mask_rle, shape=(w, h, 1))
        pred_mask = model.predict(cv2.resize(img, (256, 256)).reshape(1, 256, 256, 3)).reshape(256, 256, 1)

        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(25, 25))
        axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # change colorspace for plt
        axes[1].imshow(pred_mask[..., 0] > th_pred, cmap='gray')
        axes[2].imshow(true_mask[..., 0], cmap='gray')
        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()

In [ ]:
plot_predicts(fcn_model)

## 4. Аугментация 

In [ ]:
from albumentations import (
    CLAHE, RandomRotate90, Transpose, RandomCrop, Resize, ShiftScaleRotate, Blur, OpticalDistortion, 
    GridDistortion, HueSaturationValue, IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, 
    MedianBlur, IAAPiecewiseAffine, IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, 
    Flip, HorizontalFlip, OneOf, Compose, PadIfNeeded, LongestMaxSize, PadIfNeeded, ElasticTransform,Cutout
)

In [ ]:
# построим функцию аугментации
def strong_aug(p=1.0):
    """
    param p: вероятность применения аугментации
    """
    return Compose([
        ShiftScaleRotate(shift_limit=0.125, scale_limit=0.2, rotate_limit=10, p=0.7, border_mode=cv2.BORDER_CONSTANT),
        RandomCrop(256, 256),
        #PadIfNeeded(min_height=224, min_width=224, border_mode=cv2.BORDER_CONSTANT, p=1.0),
        #Resize(64, 64),
        #RandomRotate90(),
        ElasticTransform(1.), 
        #HorizontalFlip(),
        #Cutout(p=1.),
        #Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.3),
        OneOf([
            MotionBlur(p=.4),
            MedianBlur(blur_limit=3, p=0.3),
            Blur(blur_limit=3, p=0.3),
        ], p=0.5),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=0.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.5),
        OneOf([
            CLAHE(clip_limit=3),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.4),
        HueSaturationValue(p=0.7),  
    ],
        p=p)

In [ ]:
augmentation = strong_aug(p=1.0)

row_id = np.random.choice(val_df.shape[0], size=1, replace=False)
img_name, mask_rle = train_df.iloc[row_id[0]]
img = cv2.imread('input/train/{}'.format(img_name))
w, h, _ = img.shape
mask = rle_decode(mask_rle, shape=(w, h, 1))

data = {'image': img.astype('uint8'), 'mask': mask}
augmented = augmentation(**data)
crop_img, crop_mask = augmented["image"], augmented["mask"]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 15))
axes[0].imshow(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB))
axes[1].imshow(crop_mask[:,:,0], cmap='gray')

plt.show()

In [ ]:
# построим генератор с аугментацией и посмотрим на результат тренировки модели

def keras_generator_aug(gen_df, batch_size=4):
    while True:
        x_batch = []
        y_batch = []
        
        random_id = np.random.choice(gen_df.shape[0], size=batch_size, replace=False)
        for row_id in random_id:
            img_name, mask_rle = gen_df.iloc[row_id]
            img = cv2.imread('input/train/{}'.format(img_name))
            w, h, _ = img.shape
            mask = rle_decode(mask_rle, shape=(w, h, 1))

            img = cv2.resize(img, (256, 256))
            mask = cv2.resize(mask, (256, 256))
            
            augmentation = strong_aug(p=1.0)
            data = {'image': img.astype('uint8'), 'mask': mask}
            augmented = augmentation(**data)
            crop_img, crop_mask = augmented["image"], augmented["mask"]
            
            x_batch += [crop_img]
            y_batch += [crop_mask]

        x_batch = np.array(x_batch) / 255.  # normalize
        y_batch = np.array(y_batch)

        yield x_batch, np.expand_dims(y_batch, -1)

In [ ]:
batch_size = 4
train_gen = keras_generator_aug(train_df, batch_size)
val_gen = keras_generator_aug(val_df, batch_size)

hist = fcn_model.fit_generator(
    train_gen,
    steps_per_epoch=10,
    epochs=3,
    callbacks=callbacks,
    validation_data=val_gen,
    validation_steps=5,
    class_weight=None,
    max_queue_size=10,
    workers=6,
    use_multiprocessing=True,
    shuffle=True,
    initial_epoch=0,
    verbose=1,)

In [ ]:
plot_predicts(fcn_model)

## 5. Try Hard 

## SegNet

Усложним вариант модели на иерархический Upsampling. Задача - реализовать модель и посмотреть на результат. Примените аугментацию в генераторе для train.
<img src="https://i.ibb.co/nL3n9Br/segnet.png" width="1000">

In [ ]:
inp = Input(shape=(256, 256, 3))

conv_1_1 = Conv2D(32, (3, 3), padding='same')(inp)
conv_1_1 = Activation('relu')(conv_1_1)

conv_1_2 = Conv2D(32, (3, 3), padding='same')(conv_1_1)
conv_1_2 = Activation('relu')(conv_1_2)

pool_1 = MaxPooling2D(2)(conv_1_2)


conv_2_1 = Conv2D(64, (3, 3), padding='same')(pool_1)
conv_2_1 = Activation('relu')(conv_2_1)

conv_2_2 = Conv2D(64, (3, 3), padding='same')(conv_2_1)
conv_2_2 = Activation('relu')(conv_2_2)

pool_2 = MaxPooling2D(2)(conv_2_2)


conv_3_1 = Conv2D(128, (3, 3), padding='same')(pool_2)
conv_3_1 = Activation('relu')(conv_3_1)

conv_3_2 = Conv2D(128, (3, 3), padding='same')(conv_3_1)
conv_3_2 = Activation('relu')(conv_3_2)

pool_3 = MaxPooling2D(2)(conv_3_2)


conv_4_1 = Conv2D(256, (3, 3), padding='same')(pool_3)
conv_4_1 = Activation('relu')(conv_4_1)

conv_4_2 = Conv2D(256, (3, 3), padding='same')(conv_4_1)
conv_4_2 = Activation('relu')(conv_4_2)

pool_4 = MaxPooling2D(2)(conv_4_2)

up_1 = UpSampling2D(2, interpolation='bilinear')(pool_4)
conv_up_1_1 = Conv2D(256, (3, 3), padding='same')(up_1)
conv_up_1_1 = Activation('relu')(conv_up_1_1)

conv_up_1_2 = Conv2D(256, (3, 3), padding='same')(conv_up_1_1)
conv_up_1_2 = Activation('relu')(conv_up_1_2)


up_2 = UpSampling2D(2, interpolation='bilinear')(conv_up_1_2)
conv_up_2_1 = Conv2D(128, (3, 3), padding='same')(up_2)
conv_up_2_1 = Activation('relu')(conv_up_2_1)

conv_up_2_2 = Conv2D(128, (3, 3), padding='same')(conv_up_2_1)
conv_up_2_2 = Activation('relu')(conv_up_2_2)


up_3 = UpSampling2D(2, interpolation='bilinear')(conv_up_2_2)
conv_up_3_1 = Conv2D(64, (3, 3), padding='same')(up_3)
conv_up_3_1 = Activation('relu')(conv_up_3_1)

conv_up_3_2 = Conv2D(64, (3, 3), padding='same')(conv_up_3_1)
conv_up_3_2 = Activation('relu')(conv_up_3_2)



up_4 = UpSampling2D(2, interpolation='bilinear')(conv_up_3_2)
conv_up_4_1 = Conv2D(32, (3, 3), padding='same')(up_4)
conv_up_4_1 = Activation('relu')(conv_up_4_1)

conv_up_4_2 = Conv2D(1, (3, 3), padding='same')(conv_up_4_1)
result = Activation('sigmoid')(conv_up_4_2)


segnet_model = Model(inputs=inp, outputs=result)

adam = keras.optimizers.Adam(lr=0.1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
segnet_model.compile(adam, 'binary_crossentropy')

In [ ]:
segnet_model.summary()

In [ ]:
batch_size = 4
train_gen = keras_generator_aug(train_df, batch_size)
val_gen = keras_generator_aug(val_df, batch_size)

hist = segnet_model.fit_generator(
    train_gen,
    steps_per_epoch=100,
    epochs=5,
    callbacks=callbacks,
    validation_data=val_gen,
    validation_steps=50,
    class_weight=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    shuffle=True,
    initial_epoch=0,
    verbose=1,)

In [ ]:
plot_predicts(segnet_model)

## Vanila UNet

Пришло время для Unet. Задача аналогична - реализовать модель и посмотреть на результат. Примените аугментацию в генераторе для train.
<img src="https://i.ibb.co/wC3FxCb/unet.png" width="1000">

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D

inp = Input(shape=(256, 256, 3))

conv_1_1 = Conv2D(32, (3, 3), padding='same')(inp)
conv_1_1 = Activation('relu')(conv_1_1)

conv_1_2 = Conv2D(32, (3, 3), padding='same')(conv_1_1)
conv_1_2 = Activation('relu')(conv_1_2)

pool_1 = MaxPooling2D(2)(conv_1_2)


conv_2_1 = Conv2D(64, (3, 3), padding='same')(pool_1)
conv_2_1 = Activation('relu')(conv_2_1)

conv_2_2 = Conv2D(64, (3, 3), padding='same')(conv_2_1)
conv_2_2 = Activation('relu')(conv_2_2)

pool_2 = MaxPooling2D(2)(conv_2_2)


conv_3_1 = Conv2D(128, (3, 3), padding='same')(pool_2)
conv_3_1 = Activation('relu')(conv_3_1)

conv_3_2 = Conv2D(128, (3, 3), padding='same')(conv_3_1)
conv_3_2 = Activation('relu')(conv_3_2)

pool_3 = MaxPooling2D(2)(conv_3_2)


conv_4_1 = Conv2D(256, (3, 3), padding='same')(pool_3)
conv_4_1 = Activation('relu')(conv_4_1)

conv_4_2 = Conv2D(256, (3, 3), padding='same')(conv_4_1)
conv_4_2 = Activation('relu')(conv_4_2)

pool_4 = MaxPooling2D(2)(conv_4_2)

up_1 = UpSampling2D(2, interpolation='bilinear')(pool_4)
conc_1 = Concatenate()([conv_4_2, up_1])

conv_up_1_1 = Conv2D(256, (3, 3), padding='same')(conc_1)
conv_up_1_1 = Activation('relu')(conv_up_1_1)

conv_up_1_2 = Conv2D(256, (3, 3), padding='same')(conv_up_1_1)
conv_up_1_2 = Activation('relu')(conv_up_1_2)


up_2 = UpSampling2D(2, interpolation='bilinear')(conv_up_1_2)
conc_2 = Concatenate()([conv_3_2, up_2])
X
conv_up_2_1 = Conv2D(128, (3, 3), padding='same')(conc_2)
conv_up_2_1 = Activation('relu')(conv_up_2_1)

conv_up_2_2 = Conv2D(128, (3, 3), padding='same')(conv_up_2_1)
conv_up_2_2 = Activation('relu')(conv_up_2_2)


up_3 = UpSampling2D(2, interpolation='bilinear')(conv_up_2_2)
conc_3 = Concatenate()([conv_2_2, up_3])

conv_up_3_1 = Conv2D(64, (3, 3), padding='same')(conc_3)
conv_up_3_1 = Activation('relu')(conv_up_3_1)

conv_up_3_2 = Conv2D(64, (3, 3), padding='same')(conv_up_3_1)
conv_up_3_2 = Activation('relu')(conv_up_3_2)



up_4 = UpSampling2D(2, interpolation='bilinear')(conv_up_3_2)
conc_4 = Concatenate()([conv_1_2, up_4])
conv_up_4_1 = Conv2D(32, (3, 3), padding='same')(conc_4)
conv_up_4_1 = Activation('relu')(conv_up_4_1)

conv_up_4_2 = Conv2D(1, (3, 3), padding='same')(conv_up_4_1)
result = Activation('sigmoid')(conv_up_4_2)


unet_model = Model(inputs=inp, outputs=result)

adam = keras.optimizers.Adam(lr=0.1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
unet_model.compile(adam, 'binary_crossentropy')

In [ ]:
batch_size = 4
train_gen = keras_generator_aug(train_df, batch_size)
val_gen = keras_generator_aug(val_df, batch_size)

hist = unet_model.fit_generator(
    train_gen,
    steps_per_epoch=100,
    epochs=5,
    callbacks=callbacks,
    validation_data=val_gen,
    validation_steps=50,
    class_weight=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    shuffle=True,
    initial_epoch=0,
    verbose=1,)

In [ ]:
plot_predicts(unet_model)